<a href="https://colab.research.google.com/github/Papagoat/algo-trading-strategies/blob/main/Rayner_Teo_Trading_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Credit to [Albovibes](https://www.youtube.com/watch?v=JvfK4VBX75I)

In [ ]:
pip install yfinance

In [17]:
import pandas as pd
import numpy as np
import datetime as datetime
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import yfinance as yf
import time
import requests
import io

In [18]:
start = datetime.datetime(2010,5,18)
end = datetime.datetime.now()

# Download candlestick charts from yahoo finance

In [ ]:
df = yf.download(tickers="TSLA",start=start, end=end)

# Calculate 200 day moving average and RSI values

In [20]:
df['MA200'] = df['Adj Close'].rolling(window=200).mean()

In [21]:
df['price change'] = df['Adj Close'].pct_change()

In [22]:
df['Upmove'] = df['price change'].apply(lambda x: x if x > 0 else 0)

In [23]:
df['Downmove'] = df['price change'].apply(lambda x: abs(x) if x < 0 else 0)

In [24]:
df['Avg Up'] = df['Upmove'].ewm(span=19).mean()

In [25]:
df['Avg Down'] = df['Downmove'].ewm(span=19).mean()

In [26]:
df['RS'] = df['Avg Up']/df['Avg Down']

In [27]:
df['RSI'] = df['RS'].apply(lambda x: 100 - (100/(x+1)))

In [28]:
df.loc[(df['Adj Close'] > df['MA200']) & (df['RSI'] < 30), 'Buy'] = 'Yes'

In [29]:
df.loc[(df['Adj Close'] < df['MA200']) | (df['RSI'] > 30), 'Buy'] = 'No'

# Calculate profit and loss

In [40]:
PnL = []

for i in range(len(df) - 12):
    if "Yes" in str(df['Buy'].iloc[i]):
        for j in range(1, 11):
            if df['RSI'].iloc[i + j] > 40:
                PnL.append(df['Open'].iloc[i+j+1] - df['Open'].iloc[i+1])
                break
            elif j == 10:
                PnL.append(df['Open'].iloc[i+j+1] - df['Open'].iloc[i+1])

In [ ]:
PnL

In [ ]:
len([i for i in PnL if i > 0])

In [ ]:
len([i for i in PnL if i > 0])/len(PnL)